In [1]:
import  datasets
import  transformers
import  librosa
import  jiwer



In [2]:
from huggingface_hub import notebook_login

notebook_login()



In [3]:
import sys
List = sys.path


In [4]:
import pyarrow
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")
print(timit)


Reusing dataset timit_asr (/Users/mac/.cache/huggingface/datasets/timit_asr/clean/2.0.1/b11b576ddcccbcefa7c9f0c4e6c2a43756f3033adffe0fb686aa61043d0450ad)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})


In [5]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])


In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(timit["train"].remove_columns(["file", "audio"]))

,text
0,The overweight charmer could slip poison into anyone's tea.
1,She had your dark suit in greasy wash water all year.
2,"Once you finish greasing your chain, be sure to wash thoroughly."
3,How would you evaluate this algebraic expression?
4,She had your dark suit in greasy wash water all year.
5,Don't ask me to carry an oily rag like that.
6,"By eating yogurt, you may live longer."
7,Often they are able to get in only because the area is declining economically.
8,Don't ask me to carry an oily rag like that.
9,The pilots' heads looked ridiculously small.


In [7]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

timit = timit.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7fb710a65090> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [8]:
show_random_elements(timit["train"].remove_columns(["file", "audio"]))


,text
0,the fifth jar contains big juicy peaches
1,brush fires are common in the dry underbrush of nevada
2,doctors prescribe drugs too freely
3,pledge to participate in nevada's aquatic competition
4,when she awoke she was the ship
5,she had your dark suit in greasy wash water all year
6,clasp the screw in your left hand
7,he advised immediate hospitalization
8,the prowler wore a ski mask for disguise
9,would you allow acts of violence


In [9]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict


{'t': 0,
 'w': 1,
 'n': 2,
 'l': 3,
 'm': 4,
 'k': 5,
 'g': 6,
 'e': 7,
 'z': 8,
 'q': 9,
 'o': 10,
 "'": 11,
 'u': 12,
 'h': 13,
 'f': 14,
 'r': 15,
 'i': 16,
 ' ': 17,
 'a': 18,
 'y': 19,
 'x': 20,
 'c': 21,
 'b': 22,
 'p': 23,
 'd': 24,
 'j': 25,
 's': 26,
 'v': 27}

In [11]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]


In [12]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))


30


In [13]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [14]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [15]:
repo_name = "wav2vec2-base-timit-demo-colab"


In [16]:
tokenizer.push_to_hub(repo_name)


CommitInfo(commit_url='https://huggingface.co/sagir567/wav2vec2-base-timit-demo-colab/commit/55ae0bc264a9955f7f7be8841480a089caa82180', commit_message='Upload tokenizer', commit_description='', oid='55ae0bc264a9955f7f7be8841480a089caa82180', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


In [18]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [19]:
timit['train']['file'][0]

'/Users/mac/.cache/huggingface/datasets/downloads/extracted/86096937f792a33c316ac9ea43f48d3dc40aac8eb909880575f0f2eb2fc29ded/data/TRAIN/DR4/MMDM0/SI681.WAV'

In [20]:
# common_voice_train["audio"][0]


In [21]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)


cut a small corner off each edge


In [22]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])


Target text: bury those uniforms so they won't be found
Input array shape: (40653,)
Sampling rate: 16000


In [23]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch


In [24]:
import numpy as np
np.object = object 
timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)


/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be

In [25]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [26]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [27]:
wer_metric = load_metric("wer")


In [28]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [29]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)


/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
model.freeze_feature_extractor()


/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1919: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [31]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=False,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)


In [32]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)


In [33]:
# def ConnectButton():
#     console.log("Connect pushed"); 
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 

# setInterval(ConnectButton,60000);


In [34]:
trainer.train()


/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,3.419500,1.019881,0.664393
1000,0.499800,0.413665,0.363724
1500,0.203600,0.381278,0.318724
2000,0.116400,0.366891,0.303287
2500,0.079200,0.396502,0.295500
3000,0.065300,0.382926,0.289642
3500,0.048400,0.396885,0.283027
4000,0.038000,0.400174,0.280959


/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/Users/mac/anaconda3/envs/wav2vec2Transforrmers/lib/python3.10/site-packages/transformers/models/wav2vec2

TrainOutput(global_step=4350, training_loss=0.516459575083064, metrics={'train_runtime': 75299.3345, 'train_samples_per_second': 1.841, 'train_steps_per_second': 0.058, 'total_flos': 3.984948339456983e+18, 'train_loss': 0.516459575083064, 'epoch': 30.0})

In [53]:
notebook_login()
trainer.args.output_dir = 'wav2vec2-base-timit-demo-colab'
trainer.push_to_hub()


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sagir567/wav2vec2-base-timit-demo-colab/commit/97f09b45d6851b52185f940fe212d227d2d43db5', commit_message='End of training', commit_description='', oid='97f09b45d6851b52185f940fe212d227d2d43db5', pr_url=None, pr_revision=None, pr_num=None)

In [55]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("wav2vec2-base-timit-demo-colab")
processor = Wav2Vec2Processor.from_pretrained("wav2vec2-base-timit-demo-colab")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [56]:
processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [58]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cpu").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)


0ex [00:00, ?ex/s]

In [60]:
show_random_elements(results)


,pred_str,text
0,the fish began to leep frantically on the surface of the small eak,the fish began to leap frantically on the surface of the small lake
1,her righte hand akes whenever the barametric pressur changes,her right hand aches whenever the barometric pressure changes
2,only lawyers love milaneears,only lawyers love millionaires
3,the nearest cynegud may not be within wallkin distance,the nearest synagogue may not be within walking distance
4,bascket ball can be an entertaining sport,basketball can be an entertaining sport
5,she had your dark suit in greasy wash water all year,she had your dark suit in greasy wash water all year
6,cowsium makes bons and teach strong,calcium makes bones and teeth strong
7,it was not exactly panic they gave way to but they could not just sit there,it was not exactly panic they gave way to but they could not just sit there
8,got no busness over here on a stake out anyway,got no business over here on a stakeout anyway
9,that diagram makesense only after much studyg,that diagram makes sense only after much study


In [62]:
model.to("cpu")

with torch.no_grad():
  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cpu")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))



'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t t h e e | | b [PAD] [PAD] [PAD] [PAD] u n n g g [PAD] u l l l [PAD] [PAD] [PAD] o o [PAD] | w w [PAD] a s s s | | | p [PAD] l l [PAD] e e s s [PAD] [PAD] n n t t [PAD] l l [PAD] y y | | s s [PAD] [PAD] [PAD] i t t t [PAD] [PAD] [PAD] u u u [PAD] [PAD] [PAD] a a a a t t e e d d | | n n e e a a a r | | t h e e | | s s h h [PAD] [PAD] [PAD] [PAD] [PAD] o o o r r r [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'